In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import csv
import requests
import json
import re


In [2]:
def get_blog_url(soup):
  div_ = soup.find_all('div', attrs={'class': 'FL PR20'})
  url_list = []
  for title in div_:
    href = title.find('a')['href']
    url_list.append("https://www.moneycontrol.com/"+href)
  return url_list

In [3]:
def get_blog_content(url):
  request = requests.get(url)
  soup = BeautifulSoup(request.text, 'html.parser')
  all_scripts = soup.find_all('script', attrs = {'type':'application/ld+json'})
  raw_article_str = all_scripts[2].get_text().replace('\r\n', ' ')
  parts = re.split(r"""("[^"]*"|'[^']*')""", raw_article_str)
  parts[::2] = map(lambda s: "".join(s.split()), parts[::2]) 
  article_str = "".join(parts)
  article_str = article_str[1:]
  article_str = article_str[:-1]
  article_dict = json.loads(article_str)
  all_tags = soup.find_all('div', attrs = {'class':'tags_first_line'})
  lst_all_tags = []
  for i in all_tags:
    lst_all_tags.append(i.get_text())
  tags = lst_all_tags[0].replace('TAGS:', '')
  tags = tags.replace('\n', '')
  tags = tags.split('#')
  tags = tags[1:]
  tags = ', '.join([str(elem).strip() for elem in tags])
  article_dict['tags'] = tags
  return article_dict
  

In [4]:

def get_page_no(url, sc_id, page_no, next, year):
  request = requests.get(url)
  soup = BeautifulSoup(request.text, 'html.parser')
  all_page_no = soup.find_all('div', attrs = {'class':'pages MR10 MT15'})
  page_list = [i.text for i in all_page_no[0].find_all('a')]
  if any(map(str.isdigit, page_list[-1])):
    return int(page_list[-1]), next
  else:
    next = next + 1
    page_no = int(page_list[-2])
    url = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id="+sc_id+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
    return get_page_no(url, sc_id, page_no, next, year)

In [5]:
def save_company_data(url_ = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?", sc_id=[], page_no=1, next=0, years=[]):
  for company in sc_id:
    df = pd.DataFrame(columns=['company', 'datePublished', 'author', 'headline', 
                               'description', 'articleBody', 'tags', 'url'])
    for year in years:
      print('year: ', year)
      print('page_no: ', page_no)
      print('next: ', next)

      url = url_ + "sc_id="+company+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
      print('url: ', url)

      max_page_no, max_next = get_page_no(url, company, page_no, next, year)
      max_next = max_next + 1

      for i in range(max_next):
        for j in range((i*10)+1, (i*10)+11):
          if j <= max_page_no:
            url_list = []
            url = url_ + "sc_id="+company+"&scat=&pageno="+str(j)+"&next="+str(i)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
            request = requests.get(url)
            soup = BeautifulSoup(request.text, 'html.parser')
            url_list = get_blog_url(soup)

            for url in url_list:
              try:
                article_dict = get_blog_content(url)
                print(company)
                print(article_dict['datePublished'])
                print(article_dict['author'])
                print(article_dict['headline'])
                print(article_dict['description'])
                print(article_dict['articleBody'])
                print(article_dict['tags'])
                print(article_dict['url'])
                print('--------------------------------------------------------------------------')

                article_lst = [[company,
                                article_dict['datePublished'],
                                article_dict['author'],
                                article_dict['headline'], 
                                article_dict['description'], 
                                article_dict['articleBody'], 
                                article_dict['tags'], 
                                url]]

                df = df.append(pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline', 
                               'description', 'articleBody', 'tags', 'url']), ignore_index = True)
                
              except:
                article_lst = [[company, 'error', 'error', 'error', 'error', 'error', 'error', url]]
                df = df.append(pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline', 
                               'description', 'articleBody', 'tags', 'url']), ignore_index = True)
                
                continue
            else:
              break

    df.to_csv('content/'+company+'.csv')
    

In [8]:
save_company_data(sc_id=['RI'], years = [2022, 2021])

year:  2022
page_no:  1
next:  0
url:  https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=RI&scat=&pageno=1&next=0&durationType=Y&Year=2022&duration=1&news_type=
RI
2022-12-23T12:44:27+05:30
{'@type': 'Person', 'url': 'https://www.moneycontrol.com/author/moneycontrol-news-7285/', 'name': 'Moneycontrol News'}
RIL stock sees upgrades after Metro buyout
Securities firms say RIL&rsquo;s consumer business will be the growth driver in the years to come.
Securities firms were upbeat on Friday, December 23, about the prospects of Reliance Industries Ltd (RIL), judging its acquisition of Metro AG&rsquo;s India wholesale business to be accretive to the company&rsquo;s stakeholders.  RIL&rsquo;s subsidiary Reliance Retail Ventures Ltd (RRVL) on December 22 announced the acquisition of Metro Cash and Carry India Pvt. Ltd after signing definitive agreements to acquire 100 percent of the unit for a cash consideration of Rs 2,850 crore.  ICICI Direct has upgraded RIL&rsquo;s stock 

/var/folders/v7/fl37bh_11xn4dz8nn__9q52m0000gn/T/ipykernel_19350/2232941168.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline',


RI
2022-12-22T09:30:22+05:30
{'@type': 'Person', 'url': 'https://www.moneycontrol.com/author/moneycontrol-news-7285/', 'name': 'Moneycontrol News'}
RIL gains as investors cheer synergy benefits from Metro India acquisition
The acquisition is in line with the company's strategy of expanding its retail footprint
Share price of Reliance Industries opened higher on December 22 after the company announced the acquisition of 100 percent equity stake in Metro Cash and Carry India Pvt Ltd (Metro India) for a cash consideration of Rs 2,850 crore.  RIL started trading at Rs 2598 on the NSE, higher than the previous close of Rs 2584. The stock, with an 11 percent weightage in the Nifty 50, has gained over 7 percent in 2022 so far.  Metro India, a wholly owned subsidiary of Metro AG, started operations in India in 2003 and now runs 31 large format stores across 21 cities with about 3,500 employees. It reaches over 30 lakh B2B customers in India and 10 lakh of them are frequent buyers through its s

/var/folders/v7/fl37bh_11xn4dz8nn__9q52m0000gn/T/ipykernel_19350/2232941168.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline',


RI
2022-10-21T22:03:17+05:30
{'@type': 'Person', 'url': 'https://www.moneycontrol.com/author/moneycontrol-news-7285/', 'name': 'Moneycontrol News'}
Ten takeaways from Reliance results as conglomerate posts record Q2 for consumer businesses
The increase of 32.4 percent in gross revenue was driven by higher realisation in O2C business with sharp increase in energy prices. Revenue growth also reflected continued growth momentum across consumer businesses, the company said
Billionaire Mukesh Ambani-owned Reliance Industries on October 21 reported a record performance in consumer businesses which continued to scale new milestones every quarter, but its performance&nbsp;in the O2C (oil-to-chemical) business reflected subdued demand and weak margin environment across downstream chemical products.  Here are 10 key highlights from the quarterly earnings scorecard:  1) Profit  Reliance registered a 0.2 percent year-on-year growth in consolidated profit at Rs 15,512 crore for the quarter ended Se

...

/var/folders/v7/fl37bh_11xn4dz8nn__9q52m0000gn/T/ipykernel_19350/2232941168.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline',


RI
2021-09-11T14:56:35+05:30
{'@type': 'Person', 'url': 'https://www.moneycontrol.com/author/pti-6883/', 'name': 'PTI'}
CNG price may rise 10-11% in October: Report
The price, referred to as APM or administered rate, will rise to $3.15 per million British thermal unit (mmBtu) for the period from October 1, 2021 to March 31, 2022 from the current $1.79, ICICI Securities said  
CNG and piped cooking gas prices in cities such as Delhi and Mumbai may be hiked by 10-11 percent next month as the government-dictated gas price is set to rise by about 76 percent, ICICI Securities said in a report.  The government, using rates prevalent in gas-surplus nations, fixes the price of natural gas produced by firms such as state-owned Oil and Natural Gas Corp (ONGC) from fields given to them on nomination basis, every six months. The next review is due on October 1.  The price, referred to as APM or administered rate, will rise to $3.15 per million British thermal unit (mmBtu) for the period from Octob

/var/folders/v7/fl37bh_11xn4dz8nn__9q52m0000gn/T/ipykernel_19350/2232941168.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline',


RI
2021-09-03T12:50:09+05:30
{'@type': 'Person', 'url': 'https://www.moneycontrol.com/author/moneycontrol-news-7285/', 'name': 'Moneycontrol News'}
Read full text: Mukesh Ambani reiterates commitment to green energy
Addressing the International Climate Summit 2021, Ambani said that work was happening at a brisk pace to develop the 5,000-acre Giga Complex, which would be one of the world's largest green energy facilities, and that RIL was on track to become a net-zero carbon company by 2035.
Reliance Industries (RIL) Chairman Mukesh Ambani, on September 3, reiterated his commitment to invest Rs 75,000 crore over the next three years in green energy initiatives, including the Dhirubhai Ambani Green Energy Giga Complex in Jamnagar, as the global energy behemoth shifts its focus from hydrocarbons to renewable power.  Addressing the International Climate Summit 2021, Ambani said that work was happening at a brisk pace to develop the 5,000-acre Giga Complex, which would be one of the world's

/var/folders/v7/fl37bh_11xn4dz8nn__9q52m0000gn/T/ipykernel_19350/2232941168.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline',
/var/folders/v7/fl37bh_11xn4dz8nn__9q52m0000gn/T/ipykernel_19350/2232941168.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline',
/var/folders/v7/fl37bh_11xn4dz8nn__9q52m0000gn/T/ipykernel_19350/2232941168.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(article_lst, columns=['company', 'datePublished', 'author', 'headline',


In [9]:
df_ri = pd.read_csv("content/RI.csv")
df_ri.head()

,Unnamed: 0,company,datePublished,author,headline,description,articleBody,tags,url
0,0,RI,2022-12-23T12:44:27+05:30,"{'@type': 'Person', 'url': 'https://www.moneyc...",RIL stock sees upgrades after Metro buyout,Securities firms say RIL&rsquo;s consumer busi...,"Securities firms were upbeat on Friday, Decemb...","Buzzing Stocks, Reliance Industries",https://www.moneycontrol.com//news/buzzing-sto...
1,1,RI,2022-12-22T09:30:22+05:30,"{'@type': 'Person', 'url': 'https://www.moneyc...",RIL gains as investors cheer synergy benefits ...,The acquisition is in line with the company's ...,Share price of Reliance Industries opened high...,"Buzzing Stocks, Reliance Industries, RIL",https://www.moneycontrol.com//news/buzzing-sto...
2,2,RI,2022-10-21T22:03:17+05:30,"{'@type': 'Person', 'url': 'https://www.moneyc...",Ten takeaways from Reliance results as conglom...,The increase of 32.4 percent in gross revenue ...,Billionaire Mukesh Ambani-owned Reliance Indus...,"Chairman Mukesh Ambani, Reliance Industries, R...",https://www.moneycontrol.com//news/results/ten...
3,3,RI,error,error,error,error,error,error,https://www.moneycontrol.com//news/results/rel...
4,4,RI,error,error,error,error,error,error,https://www.moneycontrol.com//news/results/rel...
